In [1]:
import sys
sys.path.insert(0, '../../style_generation_pipeline')

In [18]:
import pandas as pd
import json
import sklearn
import glob
import pickle
from sklearn.model_selection import train_test_split
from collections import Counter

pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
from data import *

In [4]:
path='/mnt/swordfish-pool2/milad/hiatus-data/phase_1'

In [5]:
raw_data = pd.read_json(path + '/raw_data.jsonl', lines=True)

In [6]:
len(raw_data)

42734

- Here we generated first a sample of 10k documents called raw_data_sample.jsonl and its corresponding training and validation authors training_authors_sample.jsonl and valid_author_sample.jsonl --> but this wasn't ideal for clustering
- We then took the whole training and validation split of all authors resulting in 15k docuents and run the style generation, clustering and building the interpretable space

### Creating a sample of the data:

In [8]:
#sample a set to generate style features from
sample_raw_data = raw_data.sample(10000)

In [10]:
output_path = path + '/raw_data_sample.jsonl'

with open(output_path, "w") as f:
    f.write(sample_raw_data.to_json(orient='records', lines=True))

In [10]:
training_df,_,_ = get_aa_data_from_original_format(path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/data/hrs_release_08-14-23_crossGenre-combined_TA2_input',
                                              path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/groundtruth/hrs_release_08-14-23_crossGenre-combined_TA2')

In [80]:
output_path = path + '/training_candidates_and_queries.jsonl'
with open(output_path, "w") as f:
    f.write(training_df.to_json(orient='records', lines=True))

### Prepare authors for clustering:

In [14]:
sample_training_ds = pd.read_json(path + '/raw_data_sample.jsonl', lines=True)
sample_training_ds['authorID'] = sample_training_ds.authorIDs.apply(lambda x: x[0])

In [23]:
sample_docs_ids = sample_training_ds.documentID.unique()

In [24]:
# Cluster of the training and dev
training_df,_,_ = get_aa_data_from_original_format(path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/data/hrs_release_08-14-23_crossGenre-combined_TA2_input',
                                              path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/groundtruth/hrs_release_08-14-23_crossGenre-combined_TA2')

dev_df,_,_ = get_aa_data_from_original_format(path + '/train-test-dev-split/dev/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/data/hrs_release_08-14-23_crossGenre-combined_TA2_input',
                                              path + '/train-test-dev-split/dev/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/groundtruth/hrs_release_08-14-23_crossGenre-combined_TA2')

training_df.reset_index(inplace=True)
dev_df.reset_index(inplace=True)

training_df.to_json(path + "/training_authors.json")
dev_df.to_json(path + "/valid_authors.json")


training_df[training_df.documentID.isin(sample_docs_ids)].to_json(path + "/training_authors_sample.json")
dev_df[dev_df.documentID.isin(sample_docs_ids)].to_json(path + "/valid_authors_sample.json")

In [67]:
# all_authors = sample_training_ds.authorID.unique()
# training_authors, dev_authors = train_test_split(all_authors, test_size=0.20, random_state=123)
# training_df = sample_training_ds[sample_training_ds.authorID.isin(training_authors)]
# dev_df = sample_training_ds[sample_training_ds.authorID.isin(dev_authors)]

# training_df.to_json(path + "/training_authors_sample.json")
# dev_df.to_json(path + "/valid_authors_sample.json")

### Generated Style Features:

In [ ]:
!export TRANSFORMERS_CACHE="/mnt/swordfish-pool2/milad/hf-cache-new"
!export HF_DATASETS_CACHE="/mnt/swordfish-pool2/milad/hf-cache-new"
!export HF_HOME="/mnt/swordfish-pool2/milad/hf-cache-new"

!python generate_styles.py   --data-dir /mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/training_candidates_and_queries.jsonl \
                            --generator-model llama3 \
                            --shortener-model openai:gpt-3.5-turbo \
                            -style-threshold 2 \
                            --max-new-tokens 512 \
                            --device  1 2 3 4 \
                            --datadreamer_path /mnt/swordfish-pool2/milad/hiatus-data/phase_1/datadreamer

In [12]:
#df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/refined_and_aggregated_features_final.csv')
#df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explanation_10k/style_corpus.csv')
#df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explanation_10k/filtered/refined_and_aggregated_features_final.csv')
df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/filtered/refined_and_aggregated_features_final.csv')
feat_to_ling_lvl = json.load(open('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/feats_to_ling_lvl.json'))

In [13]:
df['ling_lvl'] = df.original_attribute_name.apply(lambda x: feat_to_ling_lvl[x] if x in feat_to_ling_lvl else 'other')

In [14]:
print(df.documentID.nunique())
print(df['shortend_attribute_name.v2'].nunique(), df.aggregated_name.nunique())
print(df.ling_lvl.nunique())

15820
7117 6893
4


In [15]:
df.groupby('final_attribute_name').agg({'documentID': lambda x: len(x), 'ling_lvl': lambda x: list(x)[0]}).reset_index().to_csv(path + '/explainability/final_attribute_name.csv')
df.groupby('aggregated_name').agg({'documentID': lambda x: len(x), 'ling_lvl': lambda x: list(x)[0]}).reset_index().to_csv(path + '/explainability/aggregated_name.csv')

In [19]:
g_df = df.groupby('final_attribute_name').agg({'original_attribute_name': lambda feats: {f: feats.tolist().count(f) for f in set(feats)},
                                               'shortend_attribute_name.v1': lambda feats: {f: feats.tolist().count(f) for f in set(feats)},
                                               'shortend_attribute_name.v2': lambda feats: {f: feats.tolist().count(f) for f in set(feats)},
                                               'aggregated_name': lambda feats: {f: feats.tolist().count(f) for f in set(feats)},
                                               'documentID': lambda x: len(x),
                                               'ling_lvl': lambda x: list(x)
                                    }).reset_index()

g_df['final_attribute_ling_lvl'] = g_df['ling_lvl'].apply(lambda x: Counter(x).most_common(1)[0][0])

In [20]:
g_df.sort_values('final_attribute_name').head(n=10)

final_attribute_name  \
0         abstract concepts   
1  abstract concepts rather   
2            abstract ideas   
3         abstract language   
4            abstract nouns   
5            abstract terms   
6         abstract theories   
7          academic aspects   
8         academic concepts   
9      academic disciplines   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [21]:
g_df.to_json(path + '/explainability/phase_1_style_features_corpus.json', orient='records', indent=2)

In [22]:
print(df.aggregated_name.nunique())
print(df.final_attribute_name.nunique())
g_df.final_attribute_ling_lvl.value_counts()

6893
1728


Discourse Level        518
Semantic Level         512
Syntactic Level        401
Morphological Level    297
Name: final_attribute_ling_lvl, dtype: int64

In [23]:
df['final_attribute_name'].value_counts()

diverse sentence structures                         10668
various sentence structures                          5226
complex sentence structures                          5072
simple sentence structures                           4646
technical language                                   4065
consistent verb tense                                3963
passive voice                                        3848
complex ideas                                        3797
style is formal                                      3672
technical terms                                      3601
formal tone                                          3446
transitional elements                                2944
different sentence structures                        2608
subordinate clauses                                  2424
complex concepts                                     2258
transitional phrases                                 2117
rhythmic effect                                      2054
additional inf

#### Merge the Gram2vec style feats to our LLM-based style feats:

In [29]:
# gram2vec = pd.(open('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/normalized_hiatus_gram2vec_features.csv'))
# gram2vec = {x['documentID'] : x['gram2vec_feats'] for x in gram2vec}
# gram2vec_feats = set([x for feats in gram2vec.values() for x in feats])

In [28]:
gram2vec_df.head()

,documentID,fullText,Part-of-Speech Unigram:Adjective,Part-of-Speech Unigram:Adposition (preposition or postposition),Part-of-Speech Unigram:Adverb,Part-of-Speech Unigram:Auxiliary verb,Part-of-Speech Unigram:Coordinating conjunction,Part-of-Speech Unigram:Determiner,Part-of-Speech Unigram:Interjection,Part-of-Speech Unigram:Noun,Part-of-Speech Unigram:Numeral,Part-of-Speech Unigram:Particle,Part-of-Speech Unigram:Pronoun,Part-of-Speech Unigram:Proper noun,Part-of-Speech Unigram:Punctuation,Part-of-Speech Unigram:Subordinating conjunction,Part-of-Speech Unigram:Symbol,Part-of-Speech Unigram:Verb,"Part-of-Speech Unigram:Other (foreign words, typos, abbreviations)",Part-of-Speech Unigram:Space,Part-of-Speech Bigram:Adjective followed by Adposition,Part-of-Speech Bigram:Adposition followed by Adjective,Part-of-Speech Bigram:Adjective followed by Adverb,Part-of-Speech Bigram:Adverb followed by Adjective,Part-of-Speech Bigram:Adjective followed by Auxiliary verb,Part-of-Speech Bigram:Auxiliary verb followed by Adjective,Part-of-Speech Bigram:Adjective followed by Coordinating conjunction,Part-of-Speech Bigram:Coordinating conjunction followed by Adjective,Part-of-Speech Bigram:Adjective followed by Determiner,Part-of-Speech Bigram:Determiner followed by Adjective,Part-of-Speech Bigram:Adjective followed by Interjection,Part-of-Speech Bigram:Interjection followed by Adjective,Part-of-Speech Bigram:Adjective followed by Noun,Part-of-Speech Bigram:Noun followed by Adjective,Part-of-Speech Bigram:Adjective followed by Numeral,Part-of-Speech Bigram:Numeral followed by Adjective,Part-of-Speech Bigram:Adjective followed by Particle,Part-of-Speech Bigram:Particle followed by Adjective,Part-of-Speech Bigram:Adjective followed by Pronoun,Part-of-Speech Bigram:Pronoun followed by Adjective,Part-of-Speech Bigram:Adjective followed by Proper noun,Part-of-Speech Bigram:Proper noun followed by Adjective,Part-of-Speech Bigram:Adjective followed by Punctuation,Part-of-Speech Bigram:Punctuation followed by Adjective,Part-of-Speech Bigram:Adjective followed by Subordinating conjunction,Part-of-Speech Bigram:Subordinating conjunction followed by Adjective,Part-of-Speech Bigram:Adjective followed by Symbol,Part-of-Speech Bigram:Symbol followed by Adjective,Part-of-Speech Bigram:Adjective followed by Verb,Part-of-Speech Bigram:Verb followed by Adjective,Part-of-Speech Bigram:Adjective followed by Other,Part-of-Speech Bigram:Other followed by Adjective,Part-of-Speech Bigram:Adjective followed by Space,Part-of-Speech Bigram:Space followed by Adjective,Part-of-Speech Bigram:Adposition followed by Adverb,Part-of-Speech Bigram:Adverb followed by Adposition,Part-of-Speech Bigram:Adposition followed by Auxiliary verb,Part-of-Speech Bigram:Auxiliary verb followed by Adposition,Part-of-Speech Bigram:Adposition followed by Coordinating conjunction,Part-of-Speech Bigram:Coordinating conjunction followed by Adposition,Part-of-Speech Bigram:Adposition followed by Determiner,Part-of-Speech Bigram:Determiner followed by Adposition,Part-of-Speech Bigram:Adposition followed by Interjection,Part-of-Speech Bigram:Interjection followed by Adposition,Part-of-Speech Bigram:Adposition followed by Noun,Part-of-Speech Bigram:Noun followed by Adposition,Part-of-Speech Bigram:Adposition followed by Numeral,Part-of-Speech Bigram:Numeral followed by Adposition,Part-of-Speech Bigram:Adposition followed by Particle,Part-of-Speech Bigram:Particle followed by Adposition,Part-of-Speech Bigram:Adposition followed by Pronoun,Part-of-Speech Bigram:Pronoun followed by Adposition,Part-of-Speech Bigram:Adposition followed by Proper noun,Part-of-Speech Bigram:Proper noun followed by Adposition,Part-of-Speech Bigram:Adposition followed by Punctuation,Part-of-Speech Bigram:Punctuation followed by Adposition,Part-of-Speech Bigram:Adposition followed by Subordinating conjunction,Part-of-Speech Bigram:Subordinating conjunction followed by Adposition,Part-of-Speech Bigram:Adposition followed by Symb

In [28]:
agg_df = df.groupby('documentID').agg({'final_attribute_name': lambda x: list(x)}).reset_index()
agg_df['final_attribute_name'] = agg_df.apply(lambda x: x['final_attribute_name'] + gram2vec[x['documentID']], axis=1)

In [31]:
joined_df = agg_df.explode(['final_attribute_name'])

In [32]:
joined_df.head()

,documentID,final_attribute_name
0,000d37d2-efa7-583f-b302-f0d15f552d39,long sentences
0,000d37d2-efa7-583f-b302-f0d15f552d39,complex sentences
0,000d37d2-efa7-583f-b302-f0d15f552d39,subordinate clauses
0,000d37d2-efa7-583f-b302-f0d15f552d39,visual interest
0,000d37d2-efa7-583f-b302-f0d15f552d39,pos_bigrams:AUX NUM


In [33]:
joined_df.to_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/gram2vec_and_llm_10k_based_style_features.csv')

In [35]:
joined_df.final_attribute_name.nunique()

857

### Consturcted Interpretable space:

In [79]:
#df = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/training_authors.pkl')
df = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/training_authors.pkl')

In [80]:
df.cluster_label.nunique()

157

In [81]:
#vector_to_style_dist = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/interpretable_space.pkl')
vector_to_style_dist = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/interpretable_space.pkl')

In [82]:
len(vector_to_style_dist)

157

In [83]:
clusters_and_top10_feats = {key : {y[0]: y[1] for y in sorted(x[1].items(), key=lambda i:-i[1])[:10]} for key, x in vector_to_style_dist.items()}

In [84]:
clusters_and_top10_feats

{-1: {'diverse sentence structures': 3896.937243485107,
  'simple sentence structures': 3525.239124886807,
  'consistent verb tense': 3254.6552654425795,
  'various sentence structures': 3056.1860685677843,
  'style is formal': 2991.7562013524357,
  'diverse sentence lengths': 2945.4538348194637,
  'technical language': 2888.517150400003,
  'complex sentence structures': 2830.079193336686,
  'overall rhythm': 2521.5402318724614,
  'casual language': 2237.7618424217476},
 0: {'technical language': 1009.8526756281261,
  'style is professional': 914.9718378792628,
  'style is formal': 910.2397054539705,
  'casual language': 820.9818080163645,
  'diverse sentence lengths': 660.5148621000728,
  'complex sentence structures': 483.2371057535076,
  'specialized language': 474.8632995820517,
  'simple sentence structures': 474.7968698254498,
  'diverse sentence structures': 463.2258508093673,
  'consistent verb tense': 400.4899182352229},
 1: {'better engagement': 6.9594618213043296,
  'differe

In [81]:
#json.dump(clusters_and_top10_feats, open('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability-w-gram2vec/clusters_w_top10_feats.json', 'w'))